In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error as mse,r2_score as r2, mean_absolute_error as mae , make_scorer 
import sklearn.preprocessing as pre
from sklearn.model_selection import GridSearchCV, cross_val_score, TimeSeriesSplit
#Linear Models
import sklearn.linear_model as lm
#Trees - Non-Linear
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.ensemble import AdaBoostRegressor as ada

#NeuralNet - Piecewise Linear
from sklearn.neural_network import MLPRegressor as fnn

from main import RollingWindowSplit

In [2]:
surface = pd.read_csv('/home/student/surface/Realized Variance/DatasetSPX/Dataset30_withmean.csv')

In [3]:
surface.head()

,index,date,-200.0,-195.0,-190.0,-185.0,-180.0,-175.0,-170.0,-165.0,...,vix80,vix,realVol30,realVol30_lagged,realVol30_baselag,var30,negVRP20,negVRP40,negVRP60,negVRP80
0,1996-01-04,19960104,-0.481688,-0.488486,-0.498413,-0.502789,-0.505910,-0.512287,-0.519481,-0.528043,...,0.001534,0.001561,0.000816,0.001088,0.001088,0.001696,-0.000218,-0.000335,-0.000399,-0.000446
1,1996-01-05,19960105,-0.468624,-0.474985,-0.484137,-0.487892,-0.490518,-0.496469,-0.503385,-0.511734,...,0.001430,0.001516,0.000799,0.000624,0.000624,0.001733,-0.000565,-0.000683,-0.000755,-0.000806
2,1996-01-08,19960108,-0.491648,-0.499782,-0.511191,-0.516831,-0.521025,-0.528350,-0.536287,-0.545480,...,0.001332,0.001413,0.000801,0.001346,0.000625,0.001899,0.000226,0.000114,0.000057,0.000014
3,1996-01-09,19960109,-0.552894,-0.506103,-0.506409,-0.502240,-0.497811,-0.497502,-0.498863,-0.502302,...,0.001985,0.001901,0.000984,0.001088,0.000628,0.001731,-0.000624,-0.000764,-0.000848,-0.000897
4,1996-01-10,19960110,-0.445030,-0.429575,-0.418774,-0.405147,-0.392699,-0.385341,-0.381157,-0.379901,...,0.002155,0.002211,0.001276,0.000624,0.000707,0.001490,-0.001104,-0.001312,-0.001451,-0.001531


In [4]:
surface.iloc[:,range(32,53,1)].head()

,-50.0,-45.0,-40.0,-35.0,-30.0,-25.0,-20.0,-15.0,-10.0,-5.0,...,5.0,10.0,15.0,20.0,25.0,30.0,35.0,40.0,45.0,50.0
0,-0.729244,-0.682631,-0.654935,-0.603954,-0.582463,-0.500552,-0.459661,-0.268895,0.008654,0.293555,...,-0.103371,-0.531527,-0.727050,-0.818379,-0.909441,-0.958092,-0.994752,-0.932749,-0.874313,-0.820117
1,-0.770368,-0.761074,-0.731131,-0.728772,-0.713318,-0.675094,-0.607888,-0.559876,-0.368954,-0.105730,...,0.165968,-0.284401,-0.556398,-0.714594,-0.795274,-0.834213,-0.850552,-0.884278,-0.874153,-0.819946
2,-0.789526,-0.782423,-0.791577,-0.741901,-0.711600,-0.636292,-0.611415,-0.501444,-0.318545,-0.034582,...,-0.109570,-0.474737,-0.720640,-0.859856,-0.910162,-0.911221,-0.926151,-0.932749,-0.874313,-0.820117
3,-0.654103,-0.640141,-0.579935,-0.516971,-0.517603,-0.327353,-0.262000,0.048461,0.069057,0.567946,...,0.360553,0.254118,-0.214135,-0.567721,-0.471087,-0.631862,-0.747973,-0.720941,-0.740560,-0.775995
4,-0.522807,-0.476537,-0.425725,-0.385231,-0.289686,-0.270922,-0.150425,0.025275,0.240784,0.483952,...,0.235668,-0.170467,-0.377475,-0.588265,-0.708611,-0.783132,-0.826236,-0.826986,-0.781168,-0.741373


### Estimate Model and Store Results

In [5]:
#Create an empty Dataframe to store Results
result1 = pd.DataFrame(columns=['VIX*2','Linear','Ridge','Lasso','Huber','TheilSen','RF','FNN'],index=[21,41,61,81])
result2 = pd.DataFrame(columns=['VIX*2','Linear','Ridge','Lasso','Huber','TheilSen','RF','FNN'],index=[21,41,61,81])
#Store results for our baseline model--> Use VIX*2 as a Proxy for Realized Variance
#result1.loc[81,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix80[1000:])
#result1.loc[61,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix60[1000:])
#result1.loc[41,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix40[1000:])
#result1.loc[21,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix20[1000:])
#result2.loc[81,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix80[1000:])
#result2.loc[61,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix60[1000:])
#result2.loc[41,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix40[1000:])
#result2.loc[21,'VIX*2'] = r2(surface.realVol30_lagged[1000:],surface.vix20[1000:])


In [6]:
feature_set = [[2,83,'vix80',81],[12,73,'vix60',61],[22,63,'vix40',41],[32,53,'vix20',21]]
purge=0 #5

In [7]:
'''LINEAR REGRESSION'''

Y1_true = []
Y1_hat  = []
Y2_true = []
Y2_hat  = []


for fset in feature_set:
    feature_start , feature_end , vix_code, result_index = fset
    
    
    
    for i,j in RollingWindowSplit().split_bounds(surface,1000,30, purge=purge):
        train_start, train_end = i[0], i[1]
        test_start, test_end = j[0], j[1]
        #Split Train Set
        X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
        
        Y1_train= surface.loc[train_start:train_end-1,'var30'].values   
        
        VIX_train = surface.loc[train_start:train_end-1,vix_code].values
        Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
        #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()


        #Split Test Set
        X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
        
        Y1_test= surface.loc[test_start:test_end-1,'var30'].values
        
        VIX_test = surface.loc[test_start:test_end-1,vix_code].values
        Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test



        '''REGRESSION 1'''
        #Intialize Linear Regressor
        regressor1 = lm.LinearRegression()

        #Fit the Linear Regressor
        regressor1.fit(X80_train,Y1_train)

        #Predict OOS
        Y1_true.extend(Y1_test)
        Y1_hat.extend(regressor1.predict(X80_test))


        '''REGRESSION 2'''
        #Intialize Linear Regressor
        regressor2 = lm.LinearRegression()

        #Fit the Linear Regressor
        regressor2.fit(X80_train,Y2_80_train)

        #Predict OOS
        Y2_true.extend(Y2_80_test  +  VIX_test)
        Y2_hat.extend(regressor2.predict(X80_test) + VIX_test)
       
    #Store results in the Results Dataframe
    result1.loc[result_index,'Linear'] = r2(Y1_true,Y1_hat)
    result2.loc[result_index,'Linear'] = r2(Y2_true,Y2_hat)
    

In [22]:
regressor1.coef_

array([-1.52125325e-03,  1.70568735e-03,  8.74848227e-04,  1.56730992e-03,
        2.31388791e-03, -1.99357506e-03, -3.58570430e-03,  1.30432535e-03,
       -7.20117017e-04, -4.05506195e-05,  1.20928351e-03,  4.86751952e-04,
       -6.56418826e-05, -4.68800396e-04,  2.88200395e-03, -6.01987064e-03,
       -1.36149683e-03, -1.71621536e-03,  4.26807088e-03, -5.03006258e-03,
        1.08375542e-02])

In [16]:
result2

,VIX*2,Linear,Ridge,Lasso,Huber,TheilSen,RF,FNN
21,NaN,0.221744,NaN,NaN,NaN,NaN,NaN,NaN
41,NaN,0.186199,NaN,NaN,NaN,NaN,NaN,NaN
61,NaN,0.166406,NaN,NaN,NaN,NaN,NaN,NaN
81,NaN,0.135077,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
result1

,VIX*2,Linear,Ridge,Lasso,Huber,TheilSen,RF,FNN
21,NaN,0.221744,NaN,NaN,NaN,NaN,NaN,NaN
41,NaN,0.186199,NaN,NaN,NaN,NaN,NaN,NaN
61,NaN,0.166406,NaN,NaN,NaN,NaN,NaN,NaN
81,NaN,0.135077,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
'''RIDGE REGRESSION'''

Y1_true = []
Y1_hat  = []
Y2_true = []
Y2_hat  = []
alpha1 = 1000
alpha2 = 1000

for fset in feature_set:
    feature_start , feature_end , vix_code, result_index = fset
    
    
    
    for i,j in RollingWindowSplit().split_bounds(surface,1000,30, purge=purge):
        train_start, train_end = i[0], i[1]
        test_start, test_end = j[0], j[1]
        #Split Train Set
        X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
        Y1_train= surface.loc[train_start:train_end-1,'var30'].values 
        VIX_train = surface.loc[train_start:train_end-1,vix_code].values
        Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
        #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()


        #Split Test Set
        X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
        Y1_test= surface.loc[test_start:test_end-1,'var30'].values
        VIX_test = surface.loc[test_start:test_end-1,vix_code].values
        Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test



        '''REGRESSION 1'''
        #Intialize Ridge Regressor
        regressor1 = lm.Ridge(alpha=alpha1)

        #Fit the Ridge Regressor
        regressor1.fit(X80_train,Y1_train)

        #Predict OOS
        Y1_true.extend(Y1_test)
        Y1_hat.extend(regressor1.predict(X80_test))


        '''REGRESSION 2'''
        #Intialize Ridge Regressor
        regressor2 = lm.Ridge(alpha=alpha2)

        #Fit the Ridge Regressor
        regressor2.fit(X80_train,Y2_80_train)

        #Predict OOS
        Y2_true.extend(Y2_80_test  +  VIX_test)
        Y2_hat.extend(regressor2.predict(X80_test) +  VIX_test)
   
    result1.loc[result_index,'Ridge'] = r2(Y1_true,Y1_hat)
    result2.loc[result_index,'Ridge'] = r2(Y2_true,Y2_hat)

In [29]:
'''LASSO REGRESSION'''

Y1_true = []
Y1_hat  = []
Y2_true = []
Y2_hat  = []
alpha1 = 10000
alpha2 = 10000

for fset in feature_set:
    feature_start , feature_end , vix_code, result_index = fset
    
    
    
    for i,j in RollingWindowSplit().split_bounds(surface,1000,30, purge=purge):
        train_start, train_end = i[0], i[1]
        test_start, test_end = j[0], j[1]
        #Split Train Set
        X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
        Y1_train= surface.loc[train_start:train_end-1,'var30'].values   
        VIX_train = surface.loc[train_start:train_end-1,vix_code].values
        Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
        #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()


        #Split Test Set
        X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
        Y1_test= surface.loc[test_start:test_end-1,'var30'].values
        VIX_test = surface.loc[test_start:test_end-1,vix_code].values
        Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test



        '''REGRESSION 1'''
        #Intialize Lasso Regressor
        regressor1 = lm.Lasso(alpha=alpha1)

        #Fit the Lasso Regressor
        regressor1.fit(X80_train,Y1_train)

        #Predict OOS
        Y1_true.extend(Y1_test)
        Y1_hat.extend(regressor1.predict(X80_test))


        '''REGRESSION 2'''
        #Intialize Lasso Regressor
        regressor2 = lm.Lasso(alpha=alpha2)

        #Fit the Lasso Regressor
        regressor2.fit(X80_train,Y2_80_train)

        #Predict OOS
        Y2_true.extend(Y2_80_test  +  VIX_test)
        Y2_hat.extend(regressor2.predict(X80_test) +  VIX_test)
   
    result1.loc[result_index,'Lasso'] = r2(Y1_true,Y1_hat)
    result2.loc[result_index,'Lasso'] = r2(Y2_true,Y2_hat)
    

In [20]:
'''HUBER REGRESSION'''

Y1_true = []
Y1_hat  = []
Y2_true = []
Y2_hat  = []
alpha1 = 1000
alpha2 = 1000

for fset in feature_set:
    feature_start , feature_end , vix_code, result_index = fset
    
    
    
    for i,j in RollingWindowSplit().split_bounds(surface,1000,30, purge=purge):
        train_start, train_end = i[0], i[1]
        test_start, test_end = j[0], j[1]
        #Split Train Set
        X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
        Y1_train= surface.loc[train_start:train_end-1,'var30'].values   
        VIX_train = surface.loc[train_start:train_end-1,vix_code].values
        Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
        #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()


        #Split Test Set
        X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
        Y1_test= surface.loc[test_start:test_end-1,'var30'].values
        VIX_test = surface.loc[test_start:test_end-1,vix_code].values
        Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test



        '''REGRESSION 1'''
        #Intialize Huber Regressor
        regressor1 = lm.HuberRegressor(alpha=alpha1)

        #Fit the Huber Regressor
        regressor1.fit(X80_train,Y1_train)

        #Predict OOS
        Y1_true.extend(Y1_test)
        Y1_hat.extend(regressor1.predict(X80_test))


        '''REGRESSION 2'''
        #Intialize Huber Regressor
        regressor2 = lm.HuberRegressor(alpha=alpha2)

        #Fit the Huber Regressor
        regressor2.fit(X80_train,Y2_80_train)

        #Predict OOS
        Y2_true.extend(Y2_80_test  +  VIX_test)
        Y2_hat.extend(regressor2.predict(X80_test) +  VIX_test)
   
    result1.loc[result_index,'Huber'] = r2(Y1_true,Y1_hat)
    result2.loc[result_index,'Huber'] = r2(Y2_true,Y2_hat)
    

KeyboardInterrupt: 

In [37]:
'''FEED-FORWARD NEURAL NET REGRESSION'''

Y1_true = []
Y1_hat  = []
Y2_true = []
Y2_hat  = []
alpha1 = 1
alpha2 = 1

for fset in feature_set:
    feature_start , feature_end , vix_code, result_index = fset
    
    
    
    for i,j in RollingWindowSplit().split_bounds(surface,1000,30, purge=purge):
        train_start, train_end = i[0], i[1]
        test_start, test_end = j[0], j[1]
        #Split Train Set
        X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
        Y1_train= surface.loc[train_start:train_end-1,'var30'].values   
        VIX_train = surface.loc[train_start:train_end-1,vix_code].values
        Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
        #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()


        #Split Test Set
        X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
        Y1_test= surface.loc[test_start:test_end-1,'var30'].values
        VIX_test = surface.loc[test_start:test_end-1,vix_code].values
        Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test



        '''REGRESSION 1'''
        #Intialize FNN Regressor
        regressor1 = fnn(hidden_layer_sizes=(result_index,),alpha=alpha1,max_iter=2000,early_stopping=False,shuffle=False, batch_size=100)

        #Fit the FNN Regressor
        regressor1.fit(X80_train,Y1_train)

        #Predict OOS
        Y1_true.extend(Y1_test)
        Y1_hat.extend(regressor1.predict(X80_test))


        '''REGRESSION 2'''
        #Intialize FNN Regressor
        regressor2 = fnn(hidden_layer_sizes=(result_index,),alpha=alpha2,max_iter=2000,early_stopping=False,shuffle=False,batch_size=100)

        #Fit the FNN Regressor
        regressor2.fit(X80_train,Y2_80_train)

        #Predict OOS
        Y2_true.extend(Y2_80_test  +  VIX_test)
        Y2_hat.extend(regressor2.predict(X80_test) +  VIX_test)
   
    result1.loc[result_index,'FNN'] = r2(Y1_true,Y1_hat)
    result2.loc[result_index,'FNN'] = r2(Y2_true,Y2_hat)
    

In [31]:
'''VIX'''

Y1_true = []
Y1_hat  = []
Y2_true = []
Y2_hat  = []


for fset in feature_set:
    feature_start , feature_end , vix_code, result_index = fset
    
    
    
    for i,j in RollingWindowSplit().split_bounds(surface,1000,30, purge=purge):
        train_start, train_end = i[0], i[1]
        test_start, test_end = j[0], j[1]
        #Split Train Set
        X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
        Y1_train= surface.loc[train_start:train_end-1,'var30'].values   
        VIX_train = surface.loc[train_start:train_end-1,vix_code].values
        Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
        #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()


        #Split Test Set
        X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
        Y1_test= surface.loc[test_start:test_end-1,'var30'].values
        VIX_test = surface.loc[test_start:test_end-1,vix_code].values
        Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test



        '''REGRESSION 1'''
        #Intialize VIX Regressor
        #regressor1 = lm.LinearRegression()

        #Predict OOS
        Y1_true.extend(Y1_test)
        Y1_hat.extend(VIX_test)


        '''REGRESSION 2'''
        #Intialize VIX Regressor
        #regressor2 = lm.LinearRegression()

        #Predict OOS
        Y2_true.extend(Y2_80_test  +  VIX_test)
        Y2_hat.extend(VIX_test)
       
    #Store results in the Results Dataframe
    result1.loc[result_index,'VIX*2'] = r2(Y1_true,Y1_hat)
    result2.loc[result_index,'VIX*2'] = r2(Y2_true,Y2_hat)
    

In [38]:
result1

,VIX*2,Linear,Ridge,Lasso,Huber,TheilSen,RF,FNN
21,0.2983,0.221744,0.313119,-0.010201,NaN,NaN,NaN,-434.936
41,0.346949,0.186199,0.297201,-0.010201,NaN,NaN,NaN,-579.446
61,0.372058,0.166406,0.278445,-0.010201,NaN,NaN,NaN,-736.827
81,0.386924,0.135077,0.26815,-0.010201,NaN,NaN,NaN,-1276.64


In [39]:
result2

,VIX*2,Linear,Ridge,Lasso,Huber,TheilSen,RF,FNN
21,0.2983,0.221744,0.313663,0.332714,NaN,NaN,NaN,-769.575
41,0.346949,0.186199,0.297396,0.366369,NaN,NaN,NaN,-1022.08
61,0.372058,0.166406,0.278399,0.384986,NaN,NaN,NaN,-1488.26
81,0.386924,0.135077,0.267976,0.396307,NaN,NaN,NaN,-2367.81


In [ ]:
#Robust scaling - Regression 1
robust_result1  

In [ ]:
#Robust scaling - Regression 2
robust_result2

In [ ]:
#Standard Scaling - Regression 1
standard_result1 = result1
standard_result1

In [ ]:
#Standard Scaling - Regression 2
standard_result2 = result2
standard_result2

### CROSS-VALIDATION

In [ ]:
'''RANDOM FOREST'''
cross_validation =pd.robust_result1ame(columns=['max_depth','num_estimator','r2_1','r2_2'])

Y1_true = []
Y1_hat  = []
Y2_true = []
Y2_hat  = []
n_estimator1 = [10,50,100,200]
n_estimator2 = 100
max_depth1 = [1,2,3,5]
max_depth2 = 3
counter = 0

for maxdepth in max_depth1:
    
    for n_estimator in n_estimator1:
        
        for i,j in RollingWindowSplit().split_bounds(surface[1000:4000],1000,30):
            train_start, train_end = i[0], i[1]
            test_start, test_end = j[0], j[1]
            #Split Train Set
            X80_train = surface.iloc[train_start:train_end,range(2,83,1)].values
            Y1_train= surface.loc[train_start:train_end-1,'realVol30_lagged'].values   
            VIX_train = surface.loc[train_start:train_end-1,vix_code].values
            Y2_80_train= surface.loc[train_start:train_end-1,'realVol30_lagged'].values - VIX_train
            #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()


            #Split Test Set
            X80_test = surface.iloc[test_start:test_end,range(2,83,1)].values
            Y1_test= surface.loc[test_start:test_end-1,'realVol30_lagged'].values
            VIX_test = surface.loc[test_start:test_end-1,'vix80'].values
            Y2_80_test= surface.loc[test_start:test_end-1,'realVol30_lagged'].values - VIX_test



            '''REGRESSION 1'''
            #Intialize Ridge Regressor
            regressor1 = rf(n_estimators=n_estimator,max_depth=maxdepth,criterion='mse')

            #Fit the Ridge Regressor
            regressor1.fit(X80_train,Y1_train)

            #Predict OOS
            Y1_true.extend(Y1_test)
            Y1_hat.extend(regressor1.predict(X80_test))



            '''REGRESSION 2'''
            #Intialize Ridge Regressor
            regressor2 = rf(n_estimators=n_estimator,max_depth=maxdepth,criterion='mse')

            #Fit the Ridge Regressor
            regressor2.fit(X80_train,Y2_80_train)

            #Predict OOS
            Y2_true.extend(Y2_80_test  +  VIX_test)
            Y2_hat.extend(regressor2.predict(X80_test) +  VIX_test)


        r2_1 = r2(Y1_true,Y1_hat)
        r2_2 = r2(Y2_true,Y2_hat)
        cross_validation.loc[counter] = [maxdepth,n_estimator,r2_1,r2_2]
        counter += 1
    

In [ ]:
result1_robust = pd.read_csv('/home/student/surface/Realized Variance/RESULT1_Robust.csv')
result2_robust = pd.read_csv('/home/student/surface/Realized Variance/RESULT2_Robust.csv')
result1_standard = pd.read_csv('/home/student/surface/Realized Variance/RESULT1_Standard.csv')
result2_standard = pd.read_csv('/home/student/surface/Realized Variance/RESULT2_Standard.csv')

In [ ]:
result2_robust.max()

In [ ]:
result1_robust.max()

In [ ]:
result1_robust[result1_robust.alpha==0.01]

In [ ]:
result2_robust[result2_robust.alpha==0.01]

In [ ]:
result2_standard[result2_standard.num_options==21]

In [ ]:
result2_robust[result2_robust.num_options==21]